<a href="https://colab.research.google.com/github/cmille47/CorbyRowKickoffDarty2022/blob/main/MLALG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas
import tensorflow as tf
from sklearn import model_selection
import torch

#data filepath
DATA_PATH = "/content/drive/MyDrive/MLPROJECT/notpie.csv"

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
def load_data(data_path):
    """Loads data dataset from csv file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """
    #data filepath
    filepath = data_path

    #import csv using pandas to format into dataframe
    with open(filepath, 'r') as f:
        data = pandas.read_csv(f, header=None)

    #print(data.head())
    #randomize te rows of the dataframe
    data = normalize_data(data)
    data = data.sample(frac=1)
    #load all columns except the last one into x
    x = np.array(data.iloc[:, :-1])
    y = np.array(data.iloc[:, -1])
    
    #print("Data succesfully loaded!")

    return  x,y

In [ ]:
def normalize_data(data):
    for i in range((len(data.columns)-1)):
        num = data[i].max()
        if num < 1:
            continue
        if num < 10:
            data[i] = (data[i]) / (10)
        elif num < 100:
            data[i] = (data[i]) / (100)
        elif num < 1000:
            data[i] = (data[i]) / (1000)
        else:
            print("Error in normalization! Please check!")
    return data

In [ ]:
x, y = load_data("/content/drive/MyDrive/MLPROJECT/fd3.csv")
train_ratio = 0.60
validation_ratio = 0.20
test_ratio = 0.20

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)

class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid() 
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output

#convert to tensors
training_input = torch.FloatTensor(x_train)
training_output = torch.FloatTensor(y_train)
test_input = torch.FloatTensor(x_test)
test_output = torch.FloatTensor(y_test)

input_size = training_input.size()[1] # number of features selected
hidden_size = 20 # number of nodes/neurons in the hidden layer
model = Net(input_size, hidden_size) # create the model
criterion = torch.nn.BCELoss() # works for binary classification
# without momentum parameter
#optimizer = torch.optim.SGD(model.parameters(), lr = 0.9) 
#with momentum parameter
optimizer = torch.optim.SGD(model.parameters(), lr = 0.9, momentum=0.25)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/MLPROJECT/best2.pth'))
model.train()
epochs = 5000
errors = []

for epoch in range(epochs):
    
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(training_input)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), training_output)
    errors.append(loss.item())
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()


def accuracy(model, inp, out):
  # ds is a iterable Dataset of Tensors
  # one item at a time version
  model.load_state_dict(torch.load('/content/drive/MyDrive/MLPROJECT/best2.pth'))
  guesses = []
  guess = ()
  n_correct = 0; n_wrong = 0

  for i in range(len(inp)):
    inpts = inp[i]
    target = out[i]   # float32  [0.0] or [1.0]
    with torch.no_grad():
      oupt = model(inpts)
    guess = (target, oupt.round())
    print(guess)

    # avoid 'target == 1.0'
    if guess[0]==guess[1]:
      n_correct += 1
    else:
      n_wrong += 1

  return (n_correct * 1.0) / (n_correct + n_wrong)

acc = accuracy(model,test_input,test_output)
print(f'Accuracy: {acc}')

In [ ]:
game1 = x[0]
wL = y[0]
print(x.shape)
print(game1)
print(wL)
torch.save(model.state_dict(), '/content/drive/MyDrive/MLPROJECT/.pth')
model.load_state_dict(torch.load('/content/drive/MyDrive/MLPROJECT/best.pth'))
oupt = model(torch.FloatTensor(game1))
print(oupt.item())
model.eval()

In [ ]:
def plotcharts(errors):
    errors = np.array(errors)
    plt.figure(figsize=(12, 5))
    graf02 = plt.subplot(1, 2, 1) # nrows, ncols, index
    graf02.set_title('Errors')
    plt.plot(errors, '-')
    plt.xlabel('Epochs')
    graf03 = plt.subplot(1, 2, 2)
    graf03.set_title('Tests')
    a = plt.plot(test_output.numpy(), 'yo', label='Real')
    plt.setp(a, markersize=10)
    a = plt.plot(y_pred.detach().numpy().round(), 'b+', label='Predicted')
    plt.setp(a, markersize=10)
    plt.legend(loc=7)
    plt.show()

plotcharts(errors)




In [ ]:
model.eval()
y_pred = model(test_input)
after_train = criterion(y_pred.squeeze(), test_output)
print('Test loss after training' , after_train.item())

In [ ]:
def accuracy(model, inp, out):
  # ds is a iterable Dataset of Tensors
  # one item at a time version
  n_correct = 0; n_wrong = 0

  for i in range(len(inp)):
    inpts = inp
    target = test_output   # float32  [0.0] or [1.0]
    with torch.no_grad():
      oupt = model(inpts)

    # avoid 'target == 1.0'
    if (target < 0.5)  (oupt = 0.5) and (oupt >= 0.5):
      n_correct += 1
    else:
      n_wrong += 1

  return (n_correct * 1.0) / (n_correct + n_wrong)

  print(accuracy(model))

In [ ]:
acc = accuracy(model,test_input,test_output)
print(acc)